In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d erdemtaha/cancer-data
!unzip \*.zip && rm *.zip

  0% 0.00/48.6k [00:00<?, ?B/s]
100% 48.6k/48.6k [00:00<00:00, 43.6MB/s]
Archive:  cancer-data.zip
  inflating: Cancer_Data.csv         


In [4]:
!pip install scikit-multilearn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 1.8 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from skmultilearn.adapt import MLkNN
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Step 1: Load Data (Replace 'data.csv' with your dataset)
data = pd.read_csv('/content/Cancer_Data.csv')

# Assuming 'diagnosis' is the target variable
X = data[['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
          'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean']]
y = data['diagnosis']

# Step 2: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train multiple Adaptive KNN models
n_models = 5  # Number of Adaptive KNN models

aknn_models = []
for i in range(n_models):
    aknn_model = MLkNN(k=3)  # Example parameters for Adaptive KNN
    aknn_model.fit(X_train, y_train)
    aknn_models.append(aknn_model)

# Step 4: Aggregate predictions
def aggregate_predictions(models, X_test):
    predictions = np.zeros((len(X_test), len(models)), dtype=int)
    for i, model in enumerate(models):
        predictions[:, i] = model.predict(X_test).toarray().astype(int).flatten()
    return predictions

# Step 5: Voting (Simple majority voting)
def majority_voting(predictions):
    ensemble_predictions = []
    for preds in predictions:
        counts = np.bincount(preds)
        ensemble_predictions.append(np.argmax(counts))
    return ensemble_predictions

# Step 6: Make predictions using the ensemble model
ensemble_predictions = majority_voting(aggregate_predictions(aknn_models, X_test))

# Step 7: Evaluate the ensemble model
accuracy = accuracy_score(y_test, ensemble_predictions)
precision = precision_score(y_test, ensemble_predictions, average='weighted')
recall = recall_score(y_test, ensemble_predictions, average='weighted')
f1 = f1_score(y_test, ensemble_predictions, average='weighted')

# Step 8: Display evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Step 9: Display classification report
print("\nClassification Report:\n", classification_report(y_test, ensemble_predictions))


AttributeError: 'Series' object has no attribute 'getformat'